# Federated AutoEncoder  with Flex  for Anomaly Detection

In this notebook we show how to use an Auto Encoder model for anomaly detection with federated learning using the flexible

First we do all the imports needed.

In [39]:
from flexanomalies.utils import AutoEncoder
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.aggregators_favg import aggregate_ae
from flexanomalies.pool.primitives_deepmodel import (
    build_server_model_ae,
    copy_model_to_clients_ae,
    train_ae,
    set_aggregated_weights_ae,
    weights_collector_ae,
)
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool
from flexanomalies.utils.metrics import *

Load data and define model parameters

In [40]:
model_params = {
    "epochs": 50,
    "input_dim": 9,
    "batch_size": 32,
    "neurons": [8, 4, 8],
    "hidden_act": ["relu", "relu", "relu"],
    "contamination": 0.1,
}

In [41]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat(
    "flexanomalies/datasets/data/shuttle.mat", 0.3
)

Define model

In [42]:
model = AutoEncoder(**model_params)

Once the data  is loaded, we have to federate it. For this we use the FLEX library. There are two ways to federate the data, using an IID distribution or a non IID distribution. For the IID distribution we can use the ìid_distribution function of 

FedDataDistribution. If we use a non-IID distribution, it is necessary to use a custom configuration, such as the one used in the federate_data function. For more information, go to the FLEX library workbooks, and take a look at the Federate Data with 

FLEXible notebooks.

In [43]:
flex_dataset = federate_data(5, X_train, y_train)

## Creating the federated architecture

When creating the federated architecture, we use FlexPool. Since we are running a client-server architecture, we use the client_server_architecture function. 

In [44]:
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="autoencoder_server",
    init_func=build_server_model_ae,
    model=model,
)

## Run the federated learning experiment and Evaluate

Now, we can run the federated experiment for multiple rounds using the decorators. 

Once the model is trained, we need to evaluate it at the server level.


In [ ]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_ae, pool.clients)
    pool.clients.map(train_ae)
    pool.aggregators.map(weights_collector_ae, pool.clients)
    pool.aggregators.map(aggregate_ae)
    pool.aggregators.map(set_aggregated_weights_ae, pool.servers)
output_model = pool.servers._models["autoencoder_server"]["model"]
output_model.evaluate(X_test, y_test)

In [ ]:
output_model.result_metrics_

In [47]:
save_experiments_results(
    "autoencoder",
    output_model,
    "test_autoencoder_notebook",
    model_params,
    "shuttle.mat",
    5,
    3,
    0.3,
)

## End of Notebook